In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, html, dcc

def read_tickers(filename):
    """Liest Ticker-Namen aus einer Textdatei"""
    with open(filename, 'r') as f:
        tickers = [line.strip() for line in f if line.strip()]
    return tickers

def get_portfolio_data(tickers, period='1y'):
    """Holt Daten für alle Tickers über yfinance"""
    data = {}
    failed = []
    
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            hist = stock.history(period=period)
            
            if len(hist) > 0:
                data[ticker] = {
                    'name': info.get('longName', ticker),
                    'sector': info.get('sector', 'Unknown'),
                    'industry': info.get('industry', 'Unknown'),
                    'market_cap': info.get('marketCap', 0),
                    'current_price': hist['Close'][-1],
                    'history': hist,
                    'country': info.get('country', 'Unknown')
                }
            else:
                failed.append(ticker)
        except Exception as e:
            print(f"Fehler bei {ticker}: {e}")
            failed.append(ticker)
    
    if failed:
        print(f"\nKonnte folgende Ticker nicht laden: {', '.join(failed)}")
    
    return data

def calculate_correlation_matrix(data):
    """Berechnet Korrelationsmatrix der Returns"""
    returns_df = pd.DataFrame()
    
    for ticker, info in data.items():
        returns = info['history']['Close'].pct_change()
        returns_df[ticker] = returns
    
    return returns_df.corr()

def create_sector_pie_chart(data):
    """Erstellt interaktiven Pie Chart für Sektor-Verteilung"""
    sectors = {}
    for ticker, info in data.items():
        sector = info['sector']
        sectors[sector] = sectors.get(sector, 0) + 1
    
    fig = go.Figure(data=[go.Pie(
        labels=list(sectors.keys()),
        values=list(sectors.values()),
        hole=0.3,
        marker=dict(colors=px.colors.qualitative.Set3),
        textinfo='label+percent',
        textfont_size=12
    )])
    
    fig.update_layout(
        title={
            'text': 'Portfolio-Verteilung nach Sektoren',
            'font': {'size': 20, 'color': '#2c3e50', 'family': 'Arial'},
            'x': 0.5,
            'xanchor': 'center'
        },
        height=450,
        showlegend=True,
        paper_bgcolor='#f8f9fa',
        plot_bgcolor='#f8f9fa'
    )
    
    return fig

def create_market_cap_pie_chart(data):
    """Erstellt interaktiven Pie Chart für Market Cap Kategorien"""
    categories = {'Large Cap (>$200B)': 0, 
                  'Mid Cap ($10B-$200B)': 0, 
                  'Small Cap (<$10B)': 0}
    
    for ticker, info in data.items():
        mc = info['market_cap']
        if mc > 200_000_000_000:
            categories['Large Cap (>$200B)'] += 1
        elif mc > 10_000_000_000:
            categories['Mid Cap ($10B-$200B)'] += 1
        else:
            categories['Small Cap (<$10B)'] += 1
    
    fig = go.Figure(data=[go.Pie(
        labels=list(categories.keys()),
        values=list(categories.values()),
        hole=0.3,
        marker=dict(colors=['#2ecc71', '#3498db', '#e74c3c']),
        textinfo='label+percent',
        textfont_size=12
    )])
    
    fig.update_layout(
        title={
            'text': 'Portfolio-Verteilung nach Market Cap',
            'font': {'size': 20, 'color': '#2c3e50', 'family': 'Arial'},
            'x': 0.5,
            'xanchor': 'center'
        },
        height=450,
        showlegend=True,
        paper_bgcolor='#f8f9fa',
        plot_bgcolor='#f8f9fa'
    )
    
    return fig

def create_country_pie_chart(data):
    """Erstellt interaktiven Pie Chart für Länder-Verteilung"""
    countries = {}
    for ticker, info in data.items():
        country = info['country']
        countries[country] = countries.get(country, 0) + 1
    
    fig = go.Figure(data=[go.Pie(
        labels=list(countries.keys()),
        values=list(countries.values()),
        hole=0.3,
        marker=dict(colors=px.colors.qualitative.Pastel),
        textinfo='label+percent',
        textfont_size=12
    )])
    
    fig.update_layout(
        title={
            'text': 'Portfolio-Verteilung nach Ländern',
            'font': {'size': 20, 'color': '#2c3e50', 'family': 'Arial'},
            'x': 0.5,
            'xanchor': 'center'
        },
        height=450,
        showlegend=True,
        paper_bgcolor='#f8f9fa',
        plot_bgcolor='#f8f9fa'
    )
    
    return fig

def create_sector_bar_chart(data):
    """Erstellt Bar Chart für detaillierte Sektor-Verteilung"""
    sector_counts = {}
    for info in data.values():
        sector = info['sector']
        sector_counts[sector] = sector_counts.get(sector, 0) + 1
    
    sorted_sectors = sorted(sector_counts.items(), key=lambda x: -x[1])
    sectors = [s[0] for s in sorted_sectors]
    counts = [s[1] for s in sorted_sectors]
    
    fig = go.Figure(data=[go.Bar(
        x=sectors,
        y=counts,
        marker_color=px.colors.qualitative.Set3[:len(sectors)],
        text=counts,
        textposition='auto'
    )])
    
    fig.update_layout(
        title={
            'text': 'Anzahl Positionen pro Sektor',
            'font': {'size': 20, 'color': '#2c3e50', 'family': 'Arial'},
            'x': 0.5,
            'xanchor': 'center'
        },
        xaxis_title='Sektor',
        yaxis_title='Anzahl Positionen',
        height=400,
        paper_bgcolor='#f8f9fa',
        plot_bgcolor='white',
        showlegend=False
    )
    
    return fig

def calculate_diversification_metrics(data):
    """Berechnet wichtige Diversifikations-Kennzahlen"""
    corr_matrix = calculate_correlation_matrix(data)
    
    # Durchschnittliche Korrelation
    avg_corr = corr_matrix.values[np.triu_indices_from(corr_matrix.values, k=1)].mean()
    
    # Anzahl Sektoren
    sectors = set(info['sector'] for info in data.values())
    num_sectors = len(sectors)
    
    # Anzahl Länder
    countries = set(info['country'] for info in data.values())
    num_countries = len(countries)
    
    return {
        'num_positions': len(data),
        'num_sectors': num_sectors,
        'num_countries': num_countries,
        'avg_corr': avg_corr
    }

def create_dash_app(data, metrics):
    """Erstellt Dash App mit allen Visualisierungen"""
    app = Dash(__name__)
    
    # Diversifikations-Status
    if metrics['avg_corr'] > 0.7:
        status_color = '#e74c3c'
        status_text = '⚠️ WARNUNG: Hohe Korrelation - Portfolio könnte stärker diversifiziert werden!'
    elif metrics['avg_corr'] < 0.3:
        status_color = '#2ecc71'
        status_text = '✓ Gute Diversifikation - niedrige Korrelation zwischen Assets'
    else:
        status_color = '#f39c12'
        status_text = '→ Moderate Diversifikation'
    
    app.layout = html.Div(style={'backgroundColor': '#ecf0f1', 'minHeight': '100vh', 'padding': '20px'}, children=[
        
        # Header
        html.Div(style={'textAlign': 'center', 'marginBottom': '30px'}, children=[
            html.H1('📊 Portfolio Diversifikations-Dashboard', 
                   style={'color': '#2c3e50', 'fontFamily': 'Arial', 'marginBottom': '10px'}),
            html.P('Analyse und Visualisierung deiner Portfolio-Diversifikation',
                  style={'color': '#7f8c8d', 'fontSize': '16px'})
        ]),
        
        # Metriken Cards
        html.Div(style={'display': 'flex', 'justifyContent': 'space-around', 'marginBottom': '30px', 'flexWrap': 'wrap'}, children=[
            # Card 1
            html.Div(style={'backgroundColor': 'white', 'padding': '20px', 'borderRadius': '10px', 
                           'boxShadow': '0 2px 4px rgba(0,0,0,0.1)', 'minWidth': '200px', 'margin': '10px',
                           'textAlign': 'center'}, children=[
                html.H4('Positionen', style={'color': '#7f8c8d', 'margin': '0'}),
                html.H2(str(metrics['num_positions']), style={'color': '#3498db', 'margin': '10px 0'})
            ]),
            
            # Card 2
            html.Div(style={'backgroundColor': 'white', 'padding': '20px', 'borderRadius': '10px', 
                           'boxShadow': '0 2px 4px rgba(0,0,0,0.1)', 'minWidth': '200px', 'margin': '10px',
                           'textAlign': 'center'}, children=[
                html.H4('Sektoren', style={'color': '#7f8c8d', 'margin': '0'}),
                html.H2(str(metrics['num_sectors']), style={'color': '#2ecc71', 'margin': '10px 0'})
            ]),
            
            # Card 3
            html.Div(style={'backgroundColor': 'white', 'padding': '20px', 'borderRadius': '10px', 
                           'boxShadow': '0 2px 4px rgba(0,0,0,0.1)', 'minWidth': '200px', 'margin': '10px',
                           'textAlign': 'center'}, children=[
                html.H4('Länder', style={'color': '#7f8c8d', 'margin': '0'}),
                html.H2(str(metrics['num_countries']), style={'color': '#9b59b6', 'margin': '10px 0'})
            ]),
            
            # Card 4
            html.Div(style={'backgroundColor': 'white', 'padding': '20px', 'borderRadius': '10px', 
                           'boxShadow': '0 2px 4px rgba(0,0,0,0.1)', 'minWidth': '200px', 'margin': '10px',
                           'textAlign': 'center'}, children=[
                html.H4('Ø Korrelation', style={'color': '#7f8c8d', 'margin': '0'}),
                html.H2(f'{metrics["avg_corr"]:.3f}', style={'color': '#e67e22', 'margin': '10px 0'})
            ])
        ]),
        
        # Status Banner
        html.Div(style={'backgroundColor': status_color, 'color': 'white', 'padding': '15px', 
                       'borderRadius': '10px', 'textAlign': 'center', 'marginBottom': '30px',
                       'fontSize': '18px', 'fontWeight': 'bold', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'}, 
                children=[status_text]),
        
        # Charts Row 1
        html.Div(style={'display': 'flex', 'justifyContent': 'space-between', 'marginBottom': '30px', 'flexWrap': 'wrap'}, children=[
            html.Div(style={'backgroundColor': 'white', 'borderRadius': '10px', 'padding': '20px',
                           'boxShadow': '0 2px 4px rgba(0,0,0,0.1)', 'flex': '1', 'minWidth': '400px', 'margin': '10px'}, 
                    children=[dcc.Graph(figure=create_sector_pie_chart(data), config={'displayModeBar': False})]),
            
            html.Div(style={'backgroundColor': 'white', 'borderRadius': '10px', 'padding': '20px',
                           'boxShadow': '0 2px 4px rgba(0,0,0,0.1)', 'flex': '1', 'minWidth': '400px', 'margin': '10px'}, 
                    children=[dcc.Graph(figure=create_market_cap_pie_chart(data), config={'displayModeBar': False})])
        ]),
        
        # Charts Row 2
        html.Div(style={'display': 'flex', 'justifyContent': 'space-between', 'marginBottom': '30px', 'flexWrap': 'wrap'}, children=[
            html.Div(style={'backgroundColor': 'white', 'borderRadius': '10px', 'padding': '20px',
                           'boxShadow': '0 2px 4px rgba(0,0,0,0.1)', 'flex': '1', 'minWidth': '400px', 'margin': '10px'}, 
                    children=[dcc.Graph(figure=create_country_pie_chart(data), config={'displayModeBar': False})]),
            
            html.Div(style={'backgroundColor': 'white', 'borderRadius': '10px', 'padding': '20px',
                           'boxShadow': '0 2px 4px rgba(0,0,0,0.1)', 'flex': '1', 'minWidth': '400px', 'margin': '10px'}, 
                    children=[dcc.Graph(figure=create_sector_bar_chart(data), config={'displayModeBar': False})])
        ]),
        
        # Footer
        html.Div(style={'textAlign': 'center', 'color': '#7f8c8d', 'marginTop': '40px', 'padding': '20px'}, children=[
            html.P('Portfolio Analyse Tool | Daten von Yahoo Finance', style={'margin': '0'})
        ])
    ])
    
    return app

def main(ticker_file):
    """Hauptfunktion"""
    print("Portfolio Diversifikations-Analyse")
    print("="*60)
    
    # Ticker laden
    tickers = read_tickers(ticker_file)
    print(f"\n{len(tickers)} Ticker aus {ticker_file} geladen")
    print(f"Tickers: {', '.join(tickers)}\n")
    
    # Daten holen
    print("Lade Daten von Yahoo Finance...")
    data = get_portfolio_data(tickers)
    
    if len(data) < 2:
        print("Fehler: Zu wenige valide Ticker für Analyse!")
        return
    
    print(f"\n{len(data)} Ticker erfolgreich geladen\n")
    
    # Diversifikations-Metriken
    metrics = calculate_diversification_metrics(data)
    
    print("\n" + "="*60)
    print("DIVERSIFIKATIONS-ANALYSE")
    print("="*60)
    print(f"Anzahl Positionen: {metrics['num_positions']}")
    print(f"Anzahl Sektoren: {metrics['num_sectors']}")
    print(f"Anzahl Länder: {metrics['num_countries']}")
    print(f"Durchschnittliche Korrelation: {metrics['avg_corr']:.3f}")
    print("="*60 + "\n")
    
    # Dash App erstellen und starten
    print("Starte Dashboard...")
    print("Öffne Browser: http://127.0.0.1:8050")
    app = create_dash_app(data, metrics)
    app.run(debug=True)

if __name__ == "__main__":
    # Beispiel-Verwendung
    ticker_file = "lists/filtered.txt"
    main(ticker_file)